In [2]:

import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.cluster import KMeans
import pandas as pd
from tpot import TPOTClassifier

c:\Users\paulo\anaconda3\envs\tpot_env\lib\site-packages\tpot\builtins\__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")


In [3]:
# Load the datasets
train_df = pd.read_csv(r"C:\Users\paulo\OneDrive\Documents\kaggle_competition_2_datasets\train.csv", index_col='id')

In [4]:
# Transform binary variables
train_df['Gender'] = train_df['Gender'].map({'Male': 1, 'Female': 0})
train_df['Vehicle_Damage'] = train_df['Vehicle_Damage'].map({'Yes': 1, 'No': 0})

# Ordinal Encoding for Vehicle_Age
vehicle_age_mapping = {'< 1 Year': 0, '1-2 Year': 1, '> 2 Years': 2}
train_df['Vehicle_Age'] = train_df['Vehicle_Age'].map(vehicle_age_mapping)

# Drop Driving_License due to limited variability
train_df = train_df.drop(['Driving_License'], axis=1)

In [5]:
train_df[['Gender', 'Vehicle_Damage', 'Vehicle_Age', 'Response']] = train_df[['Gender', 'Vehicle_Damage', 'Vehicle_Age', 'Response']].astype('int')

In [6]:
def optimize_dtypes(df):
    for col in df.columns:
        col_type = df[col].dtype

        if col_type == 'object':
            df[col] = df[col].astype('category')
        elif col_type.name == 'category':
            continue
        elif 'int' in str(col_type):
            col_min = df[col].min()
            col_max = df[col].max()
            if col_min >= np.iinfo(np.int8).min and col_max <= np.iinfo(np.int8).max:
                df[col] = df[col].astype('int8')
            elif col_min >= np.iinfo(np.int16).min and col_max <= np.iinfo(np.int16).max:
                df[col] = df[col].astype('int16')
            elif col_min >= np.iinfo(np.int32).min and col_max <= np.iinfo(np.int32).max:
                df[col] = df[col].astype('int32')
            elif col_min >= np.iinfo(np.int64).min and col_max <= np.iinfo(np.int64).max:
                df[col] = df[col].astype('int64')
        elif 'float' in str(col_type):
            col_max_precision = df[col].apply(lambda x: len(str(x).split('.')[1]) if '.' in str(x) else 0).max()
            if col_max_precision <= 4 and col_min >= np.finfo(np.float16).min and col_max <= np.finfo(np.float16).max:
                df[col] = df[col].astype('float16')
            elif col_max_precision <= 9 and col_min >= np.finfo(np.float32).min and col_max <= np.finfo(np.float32).max:
                df[col] = df[col].astype('float32')
            else:
                df[col] = df[col].astype('float64')
    
    return df


train_df = optimize_dtypes(train_df)

c:\Users\paulo\anaconda3\envs\tpot_env\lib\site-packages\pandas\core\dtypes\astype.py:138: RuntimeWarning: overflow encountered in cast
  return arr.astype(dtype, copy=True)


In [7]:
# train_df[['Gender', 'Vehicle_Damage', 'Vehicle_Age', 'Response']] = train_df[['Gender', 'Vehicle_Damage', 'Vehicle_Age', 'Response']].astype('category')

In [8]:
# train_df.info()

In [9]:
# Specify the target and features
target = 'Response'
features = train_df.drop(columns=[target])
X = features
y = train_df[target]

# Ensure target is 1-D array
y = y.values

# Replace infinities with NaNs
X.replace([np.inf, -np.inf], np.nan, inplace=True)

# Drop rows with NaNs
X.dropna(inplace=True)
y = y[X.index]

In [10]:
# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Initialize the TPOT classifier
tpot = TPOTClassifier(verbosity=2, generations=5, population_size=20, scoring='roc_auc', random_state=42, n_jobs=-1)

# Train the model
tpot.fit(X_train, y_train)

# Evaluate the model
y_pred = tpot.predict(X_test)
roc_auc = roc_auc_score(y_test, y_pred)
print(f'ROC AUC Score: {roc_auc}')

# Export the best model pipeline
tpot.export('tpot_best_model_pipeline.py')

Version 0.12.1 of tpot is outdated. Version 0.12.2 was released Friday February 23, 2024.


                                                                    
                                                                    
TPOT closed during evaluation in one generation.
                                                                    
                                                                    
TPOT closed prematurely. Will use the current best pipeline.


RuntimeError: A pipeline has not yet been optimized. Please call fit() first.